In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime,timedelta
import inflect
from tqdm import tqdm

In [2]:
def date_range(start, end):
    r = (end+timedelta(days=1)-start).days
    return [start+timedelta(days=i) for i in range(r)]

In [3]:
start_date = "2021-11-01"
end_date = "2022-02-21"
end = datetime.strptime(end_date, '%Y-%m-%d')
start = datetime.strptime(start_date, '%Y-%m-%d')
dateList = date_range(start, end)

In [4]:
sources = pd.read_csv("sources.csv")

In [5]:
sources['Parameters'] = sources.Parameters.apply(lambda x: x[1:-1].split(','))

In [6]:
# sources.head()

In [7]:
Date = []
State = []
Parameter = []
Source = []
ArtifactURI = []
ExtractTime = []
ArtifactFileFormat = []
# params = ["Confirmed","Recovered","Deceased","Others/Migrated","Tested","Vaccination1","Vaccination2","Vaccination3"]

In [8]:
fs = pd.read_csv("fileStatus.csv")

In [9]:
# fs[(fs["Date"] == "2021-11-01") & (fs["State"] == "TT")]["myGovFlag"].item()

In [10]:
for date in tqdm(dateList):
    for idx in sources.index:
        p = inflect.engine()
        for parameter in sources["Parameters"][idx]:
            Date.append(str(date.date()))
            State.append(sources["StateName"][idx])
            Parameter.append(parameter)
            if sources["StateDataSourceType"][idx] == "Image(Twitter)":
                ArtifactURI.append("https://github.com/incovid19/INPUT/"+str(date.date())+"/"+sources["StateCode"][idx]+".jpeg")
            else:
                ArtifactURI.append("https://github.com/incovid19/INPUT/"+str(date.date())+"/"+sources["StateCode"][idx]+"."+sources["StateDataSourceType"][idx])
            if parameter in ["Confirmed","Recovered","Deceased","Others/Migrated","Tested"]:
                try:
                    if fs[(fs["Date"] == str(date.date())) & (fs["State"] == sources["StateCode"][idx])]["myGovFlag"].item() == "No":
                        if sources["StateDataSourceType"][idx] == "pdf":
                            if sources["StateCode"][idx] == "PB":
                                Source.append('https://nhm.punjab.gov.in/advertisements/Media_Bulletin/Media%20Bulletin%20COVID-19%20' + date.strftime("%d-%m-%Y") + '.pdf')
                            elif sources["StateCode"][idx] == "ML":
                                Source.append('https://meghalaya.gov.in/sites/default/files/announcement/Daily_Covid_' + date.strftime("%d_%b_%Y.pdf"))
                            elif sources["StateCode"][idx] == "UT":
                                Source.append('https://health.uk.gov.in/files/' + date.strftime("%Y.%m.%d_Health_Bulletin.pdf"))
                            elif sources["StateCode"][idx] == "WB":
                                url = 'https://www.wbhealth.gov.in/uploaded_files/corona/WB_DHFW_Bulletin_' + p.ordinal(
                                        date.day).upper() + '_' + date.strftime("%b").upper() + '_REPORT_FINAL.pdf'
                                Source.append(url)
                            elif sources["StateCode"][idx] == "TN":
                                url = 'https://stopcorona.tn.gov.in/wp-content/uploads/2020/03/Media-Bulletin-' + date.strftime(
                                        "%d-%m-%y") + '-COVID-19.pdf'
                                Source.append(url)
                            elif sources["StateCode"][idx] == "HR":
                                url = 'http://nhmharyana.gov.in/WriteReadData/userfiles/file/CoronaVirus/Daily%20Bulletin%20of%20COVID%2019%20as%20on%20' + date.strftime(
                                        "%d-%m-%Y") + '.pdf'
                                Source.append(url)
                            else:
                                Source.append(sources["StateDataURL"][idx])
                        else:
                            Source.append(sources["StateDataURL"][idx])
                    else:
                        Source.append("https://www.mygov.in/covid-19")
                except:
                    Source.append("https://www.mygov.in/covid-19")
            else:
                Source.append("https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports")
            df = pd.read_csv("../RAWCSV/"+str(date.date())+"/"+sources["StateCode"][idx]+"_final.csv")
            ExtractTime.append(df["last_updated"][0])
            ArtifactFileFormat.append(sources["StateDataSourceType"][idx])

100%|██████████| 113/113 [02:50<00:00,  1.51s/it]


In [11]:
df_rdf = pd.DataFrame(list(zip(Date,State,Parameter,Source,ArtifactURI,ExtractTime,ArtifactFileFormat)))

In [12]:
df_rdf = df_rdf.rename(columns={0:"Date",1:"State",2:"Parameter",3:"Source",4:"ArtifactURI",5:"ExtractTime",6:"ArtifactFileFormat"})

In [13]:
df_rdf.to_csv("states_rdf.csv",index=False)

In [14]:
df_dist = pd.read_csv("../StateDistricts.csv")

In [15]:
df_dist.head()

,State,District
0,Andaman and Nicobar Islands,Nicobars
1,Andaman and Nicobar Islands,North and Middle Andaman
2,Andaman and Nicobar Islands,South Andaman
3,Andaman and Nicobar Islands,Unknown
4,Andhra Pradesh,Foreign Evacuees


In [16]:
df_rdf

,Date,State,Parameter,Source,ArtifactURI,ExtractTime,ArtifactFileFormat
0,2021-11-01,India,Confirmed,https://www.mygov.in/covid-19,https://github.com/incovid19/INPUT/2021-11-01/...,2021-11-12 19:36:55.656023,html
1,2021-11-01,India,Recovered,https://www.mygov.in/covid-19,https://github.com/incovid19/INPUT/2021-11-01/...,2021-11-12 19:36:55.656023,html
2,2021-11-01,India,Deceased,https://www.mygov.in/covid-19,https://github.com/incovid19/INPUT/2021-11-01/...,2021-11-12 19:36:55.656023,html
3,2021-11-01,India,Others/Migrated,https://www.mygov.in/covid-19,https://github.com/incovid19/INPUT/2021-11-01/...,2021-11-12 19:36:55.656023,html
4,2021-11-01,India,Tested,https://www.mygov.in/covid-19,https://github.com/incovid19/INPUT/2021-11-01/...,2021-11-12 19:36:55.656023,html
...,...,...,...,...,...,...,...
33443,2022-02-21,Puducherry,Others/Migrated,https://www.mygov.in/covid-19,https://github.com/incovid19/INPUT/2022-02-21/...,2022-02-21 22:56:17.013864,Image(Twitter)
33444,2022-02-21,Puducherry,Tested,https://www.mygov.in/covid-19,https://github.com/incovid19/INPUT/2022-02-21/...,2022-02-21 22:56:17.013864,Image(Twitter)
33445,2022-02-21,Puducherry,Vaccination1,https://api.cowin.gov.in/api/v1/reports/v2/get...,https://github.com/incovid19/INPUT/2022-02-21/...,2022-02-21 22:56:17.013864,Image(Twitter)
33446,2022-02-21,Puducherry,Vaccination2,https://api.cowin.gov.in/api/v1/reports/v2/get...,https://github.com/incovid19/INPUT/2022-02-21/...,2022-02-21 22:56:17.013864,Image(Twitter)


In [17]:
df_dist.merge(df_rdf,left_on="State",right_on="State").to_csv("District_rdf.csv",index=False)